In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

# G-scores for each training setup
rgd = np.array([
    0.9108, 0.6784, 0.6481, 0.7137,  # ImageNet: Color, Noise, Frequency, Texture
    0.9578, 1.0361, 1.2573, 1.0605,  # Face: Color, Noise, Frequency, Texture
    0.6120, 0.7900, 0.9950, 0.8057   # Bedroom: Color, Noise, Frequency, Texture
])

rdg = np.array([
    0.9883, 1.0492, 1.2505, 1.0045,  # ImageNet: Color, Noise, Frequency, Texture
    0.9509, 0.7575, 0.6720, 0.8058,  # Face: Color, Noise, Frequency, Texture
    0.7220, 1.0762, 1.0015, 1.0195   # Bedroom: Color, Noise, Frequency, Texture
])

rdgh = np.array([
    0.9980, 0.9810, 0.9803, 0.9954,  # ImageNet: Color, Noise, Frequency, Texture
    1.0014, 0.9979, 1.0099, 1.0024,  # Face: Color, Noise, Frequency, Texture
    1.0023, 1.0173, 1.0372, 1.0178   # Bedroom: Color, Noise, Frequency, Texture
])

# Paired t-test
t_stat, p_two_sided = stats.ttest_rel(rdg, rgd)
p_one_sided = p_two_sided / 2 if t_stat > 0 else 1 - p_two_sided / 2

# Create summary table
summary_table = pd.DataFrame({
    "Training Setup": ["Real+GAN ⇒ Diffusion", "Real+Diffusion ⇒ GAN"],
    "Mean": [np.mean(rgd), np.mean(rdg)],
    "Std Dev": [np.std(rgd, ddof=1), np.std(rdg, ddof=1)],
    "One-sided p-value": [p_one_sided, p_one_sided]
})

summary_table = summary_table.round(6)
print("\n=== Summary Training Setup Table ===")
print(summary_table.to_string(index=False))


=== Summary Training Setup Table ===
      Training Setup     Mean  Std Dev  One-sided p-value
Real+GAN ⇒ Diffusion 0.872117 0.196863           0.237205
Real+Diffusion ⇒ GAN 0.941492 0.169018           0.237205


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import friedmanchisquare, wilcoxon

# G-scores for each feature type across 3 datasets × 2 setups
g_scores = {
    'Color':     np.array([0.9108, 0.9883, 0.9835, 0.9509, 0.6120, 0.7220]),
    'Noise':     np.array([0.6784, 1.0492, 1.0361, 0.7575, 0.7900, 1.0762]),
    'Frequency': np.array([0.6481, 1.2505, 1.2573, 0.6720, 0.9950, 1.0015]),
    'Texture':   np.array([0.7137, 1.0045, 1.0605, 0.8058, 0.8057, 1.0195])
}

# Friedman test for repeated measures
friedman_stat, friedman_p = friedmanchisquare(
    g_scores['Color'], g_scores['Noise'],
    g_scores['Frequency'], g_scores['Texture']
)

# Pairwise Wilcoxon tests (we use average of p-values for inclusion in summary)
features = list(g_scores.keys())
pval_summary = []
for f in features:
    pvals = []
    for other in features:
        if f != other:
            _, p = wilcoxon(g_scores[f], g_scores[other])
            pvals.append(p)
    mean_pval = np.mean(pvals)
    pval_summary.append(mean_pval)

# Custom summary table with added mean p-value
summary_custom = pd.DataFrame({
    'Data': ['color', 'noise', 'frequency', 'texture & structure'],
    'Mean': [
        g_scores['Color'].mean(),
        g_scores['Noise'].mean(),
        g_scores['Frequency'].mean(),
        g_scores['Texture'].mean()
    ],
    'Std Dev': [
        g_scores['Color'].std(ddof=1),
        g_scores['Noise'].std(ddof=1),
        g_scores['Frequency'].std(ddof=1),
        g_scores['Texture'].std(ddof=1)
    ],
    'Avg p-value': pval_summary
})

summary_custom = summary_custom.round(6)
print(summary_custom.to_string(index=False))

               Data     Mean  Std Dev  Avg p-value
              color 0.861250 0.156905     0.656250
              noise 0.897900 0.175109     0.760417
          frequency 0.970733 0.266556     0.437500
texture & structure 0.901617 0.143819     0.708333


In [4]:
import pandas as pd

# 1) Raw data
data = {
    'Dataset': ['Bedroom', 'ImageNet', 'Face'],
    'HSV':  [0.6848, 0.8214, 0.8295],
    'RGB':  [0.2973, 0.1429, 0.1350],
    'YCrCb':[0.0179, 0.0357, 0.0355],
}
df = pd.DataFrame(data)

# 2) Compute overall average row
avg_values = df[['HSV', 'RGB', 'YCrCb']].mean()
avg_row = pd.DataFrame([{
    'Dataset': 'Overall Average',
    'HSV':      avg_values['HSV'],
    'RGB':      avg_values['RGB'],
    'YCrCb':    avg_values['YCrCb'],
}])
df = pd.concat([df, avg_row], ignore_index=True)

# 3) Convert to percentages (one decimal) and add Top Modality
for col in ['HSV', 'RGB', 'YCrCb']:
    df[col] = (df[col] * 100).round(1)

df['Top Modality'] = df[['HSV','RGB','YCrCb']].idxmax(axis=1)

# 4) Rename columns for clarity
df = df.rename(columns={
    'HSV': 'HSV (%)',
    'RGB': 'RGB (%)',
    'YCrCb': 'YCrCb (%)'
})

print(df.to_string(index=False))

        Dataset  HSV (%)  RGB (%)  YCrCb (%) Top Modality
        Bedroom     68.5     29.7        1.8          HSV
       ImageNet     82.1     14.3        3.6          HSV
           Face     83.0     13.5        3.6          HSV
Overall Average     77.9     19.2        3.0          HSV


In [5]:
import pandas as pd

# 1) Raw data
data = {
    'Dataset': ['Bedroom', 'ImageNet', 'Face'],
    'Local Noise':  [0.2895, 0.2672, 0.3067],
    'Wavelet Noise':  [0.7105, 0.7328, 0.6933]
}
df = pd.DataFrame(data)

# 2) Compute overall average row
avg_values = df[['Local Noise', 'Wavelet Noise']].mean()
avg_row = pd.DataFrame([{
    'Dataset': 'Overall Average',
    'Local Noise': avg_values['Local Noise'],
    'Wavelet Noise': avg_values['Wavelet Noise'],
}])

df = pd.concat([df, avg_row], ignore_index=True)

# 3) Convert to percentages (one decimal) and add Top Modality
for col in ['Local Noise', 'Wavelet Noise']:
    df[col] = (df[col] * 100).round(1)

df['Top Modality'] = df[['Local Noise','Wavelet Noise']].idxmax(axis=1)

# 4) Rename columns for clarity
df = df.rename(columns={
    'Local Noise': 'Local Noise (%)',
    'Wavelet Noise': 'Wavelet Noise (%)'
})

print(df.to_string(index=False))

        Dataset  Local Noise (%)  Wavelet Noise (%)  Top Modality
        Bedroom             29.0               71.0 Wavelet Noise
       ImageNet             26.7               73.3 Wavelet Noise
           Face             30.7               69.3 Wavelet Noise
Overall Average             28.8               71.2 Wavelet Noise


In [6]:
import pandas as pd

# 1) Raw data
data = {
    'Dataset':                  ['Bedroom', 'ImageNet', 'Face'],
    'Slope':                    [0.5263, 0.5079, 0.5210],
    'High-Frequency Ratio':     [0.4737, 0.4921, 0.4790]
}
df = pd.DataFrame(data)

# 2) Compute overall average row
avg_values = df[['Slope', 'High-Frequency Ratio']].mean()
avg_row = pd.DataFrame([{
    'Dataset': 'Overall Average',
    'Slope': avg_values['Slope'],
    'High-Frequency Ratio': avg_values['High-Frequency Ratio'],
}])

df = pd.concat([df, avg_row], ignore_index=True)

# 3) Convert to percentages (one decimal) and add Top Modality
for col in ['Slope', 'High-Frequency Ratio']:
    df[col] = (df[col] * 100).round(1)

df['Top Modality'] = df[['Slope','High-Frequency Ratio']].idxmax(axis=1)

# 4) Rename columns for clarity
df = df.rename(columns={
    'Slope': 'Slope (%)',
    'High-Frequency Ratio': 'High-Frequency Ratio (%)'
})

print(df.to_string(index=False))

        Dataset  Slope (%)  High-Frequency Ratio (%) Top Modality
        Bedroom       52.6                      47.4        Slope
       ImageNet       50.8                      49.2        Slope
           Face       52.1                      47.9        Slope
Overall Average       51.8                      48.2        Slope


In [7]:
import pandas as pd

# 1) Raw data
data = {
    'Dataset':             ['Bedroom', 'ImageNet', 'Face'],
    'VGG channels':        [0.5640, 0.6581, 0.7794],
    'LPB histogram':       [0.3388, 0.2406, 0.1502],
    'GLCM props':          [0.0972, 0.1012, 0.0705]
}
df = pd.DataFrame(data)

# 2) Compute overall average row
avg_values = df[['VGG channels', 'LPB histogram', 'GLCM props']].mean()
avg_row = pd.DataFrame([{
    'Dataset': 'Overall Average',
    'VGG channels': avg_values['VGG channels'],
    'LPB histogram': avg_values['LPB histogram'],
    'GLCM props': avg_values['GLCM props'],
}])

df = pd.concat([df, avg_row], ignore_index=True)

# 3) Convert to percentages (one decimal) and add Top Modality
for col in ['VGG channels', 'LPB histogram', 'GLCM props']:
    df[col] = (df[col] * 100).round(1)

df['Top Modality'] = df[['VGG channels','LPB histogram','GLCM props']].idxmax(axis=1)

# 4) Rename columns for clarity
df = df.rename(columns={
    'VGG channels': 'VGG channels (%)',
    'LPB histogram': 'LPB histogram (%)',
    'GLCM props': 'GLCM props (%)'
})

print(df.to_string(index=False))

        Dataset  VGG channels (%)  LPB histogram (%)  GLCM props (%) Top Modality
        Bedroom              56.4               33.9             9.7 VGG channels
       ImageNet              65.8               24.1            10.1 VGG channels
           Face              77.9               15.0             7.0 VGG channels
Overall Average              66.7               24.3             9.0 VGG channels
